### Las Losowy

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load

In [2]:
def loadData(file_name):
    return pd.read_csv("../data/processed/" + file_name)

train_data = loadData("train.csv")
test_data = loadData("test.csv")

In [3]:
target = "time"
features = train_data.columns.tolist()
features.remove(target)

In [4]:
train_data_X = train_data[features]
train_data_Y = train_data[target]

### Strojenie modelu

In [5]:
random_grid = {
    "n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 100)],
    "max_features": ['auto', 'sqrt'],
    "max_depth": [int(x) for x in np.linspace(1, 100, num = 50)],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

In [6]:
randomForest = RandomForestRegressor()
randomForest_Randomizer = RandomizedSearchCV(
    estimator = randomForest, 
    param_distributions = random_grid, 
    n_iter = 1000, 
    cv = 3, 
    verbose = 2, 
    random_state = 42, 
    n_jobs = -1)

Zostało przetestowane $1000$ kombinacji parametrów, w 3-krotnej strategii podziału z walidacją krzyżową. 

In [7]:
# randomForest_Randomizer.fit(train_data_X, train_data_Y)

Ze względu na dużą ilość kombinacji i długi czas wykonywania obliczeń testowanie parametrów odbyło się poza notesem.

Najlepsza kombinacja:

```
{'n_estimators': 272, 
'min_samples_split': 2, 
'min_samples_leaf': 4, 
'max_features': 'sqrt', 
'max_depth': 41, 
'bootstrap': True}
```
Wynik: 0.7009982704409756

### Trenowanie modelu

#### Na hiperparamtrach domyślnych 

In [8]:
model_default = RandomForestRegressor(random_state = 42)
model_default.fit(train_data_X, train_data_Y)

RandomForestRegressor(random_state=42)

#### Na hiperparametrach dostrojonych

In [9]:
model_tuned = RandomForestRegressor(
    random_state = 42,
    n_estimators = 272, 
    min_samples_split = 2,
    min_samples_leaf = 4,
    max_features = 'sqrt',
    max_depth = 41,
    bootstrap = True)

model_tuned.fit(train_data_X, train_data_Y)

RandomForestRegressor(max_depth=41, max_features='sqrt', min_samples_leaf=4,
                      n_estimators=272, random_state=42)

### Porównanie modeli

#### Model domyślny

In [10]:
expected_values = test_data[target].to_numpy()

In [11]:
predicted_values_default = model_default.predict(test_data[features])

In [12]:
MSE_default = (np.square(expected_values - predicted_values_default)).mean()
MSE_default

122.06880514140224

#### Model dostrojony

In [13]:
predicted_values_tuned = model_tuned.predict(test_data[features])

In [14]:
MSE_tuned = (np.square(expected_values - predicted_values_tuned)).mean()
MSE_tuned

110.70194167683361

#### Stosunek błędów średniokwadratowych

In [15]:
MSE_default / MSE_tuned

1.1026798924426395

Model ze strojonymi parametrami uzyskał $10\%$ mniejszy błąd średniokwadratowy na zbiorze testowym

### Zapisanie gotowego modelu do pliku

Ze względu na mniejszy błąd średniokwadratowy modelu dostrajanego to on został wybrany 

In [16]:
dump(model_tuned, "../models/random_forest_model.joblib")

['../models/random_forest_model.joblib']